In [1]:
import pandas as pd
#import janitor as jn

In [2]:
pd.set_option('max_columns', None)

## Charger les données

In [3]:
data = pd.read_pickle("../../data/processed/maison.pkl")
#data.to_pickle("../../data/processed/maison.pkl")

In [4]:
# Afficher la table initiale
print(data.info(null_counts=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315650 entries, 0 to 315649
Data columns (total 9 columns):
 #   Column                     Non-Null Count   Dtype   
---  ------                     --------------   -----   
 0   valeur_fonciere            315650 non-null  float64 
 1   adresse_code_voie          315650 non-null  category
 2   code_postal                315650 non-null  category
 3   code_commune               315650 non-null  category
 4   code_departement           315650 non-null  category
 5   surface_reelle_bati        315650 non-null  float64 
 6   nombre_pieces_principales  315650 non-null  float64 
 7   code_nature_culture        315650 non-null  category
 8   surface_terrain            315650 non-null  float64 
dtypes: category(5), float64(4)
memory usage: 14.1 MB
None


## Traitement des données

In [5]:
num_features = data.select_dtypes("float").columns
categ_features = data.select_dtypes("category").columns

In [6]:
data[categ_features] = data[categ_features].astype("str")

In [7]:
#data["code_postal"] = data["code_postal"].astype("str")
cat_f_to_keep = ["code_postal"]
cat_f_to_ignore = categ_features[categ_features!=cat_f_to_keep[0]]

target = 'valeur_fonciere'

num_f_to_keep = list(num_features[num_features!=target])

In [ ]:
sub_pc = (data["code_postal"].value_counts().cumsum()<40000)
list_sub_pc = list(sub_pc.index[sub_pc])
df = data[data["code_postal"].isin(list_sub_pc)]

df = df[(df["valeur_fonciere"]>30000) & (df["valeur_fonciere"]<300000)]
print(data.shape, df.shape)

In [8]:
df = data[(data["valeur_fonciere"]>30000) & (data["valeur_fonciere"]<300000)]
print(data.shape, df.shape)

(315650, 9) (243287, 9)


## Pycaret - Regression

PROB : pour la scission des données, faire proportionnel dans chaque classe. Mais il y a peut-etre des classes manquantes dans les données d'entrainement que l'on a seulement dans l'ensemble de test.

In [9]:
from pycaret.regression import *

In [ ]:
exp_reg101 = setup(data = df, target = target, ignore_features=cat_f_to_ignore, categorical_features=cat_f_to_keep,
                   remove_outliers=True, #remove_multicollinearity = True,# multicollinearity_threshold = 0.6,
                   #polynomial_features = True, #feature_interaction=True, #trigonometry_features = True, 
                   normalize = True, normalize_method="robust", transformation = True, transform_target = True, verbose=True, session_id=123)
# normalize_method='robust'

IntProgress(value=0, description='Processing: ', max=13)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,20:48:22
Status,. . . . . . . . . . . . . . . . . .,Preparing Data for Modeling
ETC,. . . . . . . . . . . . . . . . . .,Calculating ETC


Text(value="Following data types have been inferred automatically, if they are correct press enter to continue…

,Data Type
valeur_fonciere,Label
code_postal,Categorical
surface_reelle_bati,Numeric
nombre_pieces_principales,Numeric
surface_terrain,Numeric


In [ ]:
# exp_reg101 = setup(data = data, target = 'valeur_fonciere', remove_outliers=True,
#                    ignore_features=cat_features, create_clusters=True, polynomial_features=True, feature_ratio=True, 
#                    profile=True, session_id=123) 
# exp_reg101 = setup(data = data, target = target, remove_outliers=True, categorical_features=cat_f_tokeep,
#                    numeric_features=num_f_tokeep, ignore_features=cat_f_toignore,
#                    session_id=123) 

In [ ]:
compare_models(exclude=["catboost", "svm", "et", "tr", "rf", "ransac", "gbr","ada"], fold=5, n_select=3)

In [ ]:
models()

## Restes

In [ ]:
interpret_model(xg_reg)

In [ ]:
xg_reg = automl(use_holdout=False)
xg_reg

In [ ]:
plot_model(xg_reg, plot = 'residuals')

In [ ]:
pull()

In [ ]:
xg_reg = create_model("xgboost")
tune_xg_reg = tune_model(xg_reg, n_iter=100, fold=10, choose_better=True)
print(tune_xg_reg)

In [ ]:
tune_xg_reg2 = tune_model(tune_xg_reg2, n_iter=10, fold=10, choose_better=True)
print(tune_xg_reg)

In [ ]:
print(tune_xg_reg2)

In [ ]:
finalize_model(xg_reg)

In [ ]:
save_model(xg_reg,'../../models/final_xg_reg_for_streamlit_test')

In [ ]:
predict_model(xg_reg)

In [ ]:
interpret_model(xg_reg)

In [ ]:
rf_reg = create_model("rf")
tune_rf_reg = tune_model(rf_reg, n_iter=100, fold=10, choose_better=True)
print(tune_rf_reg)

In [ ]:
compare_models(whitelist=["catboost", "svm"], fold=5, n_select=2)

In [ ]:
compare_models(blacklist=["catboost"], fold=5, n_select=1)

In [ ]:
catboost_reg = create_model("catboost")

In [ ]:
tune_huber_reg = tune_model(huber_reg, n_iter=100, fold=10, choose_better=True)
print(tune_huber_reg)

In [ ]:
#evaluate_model(tune_huber_reg)

In [ ]:
interpret_model(tune_huber_reg)

In [ ]:
exp_reg102 = setup(data = data, target = 'valeur_fonciere', remove_outliers=True,
                   ignore_features=cat_features, polynomial_features=True,
                   session_id=124) 

In [ ]:
compare_models(blacklist=["catboost","svm"], fold=5, n_select=1)

In [ ]:
lr_reg = create_model("lr")

In [ ]:
tune_lr_reg = tune_model(lr_reg, n_iter=100, fold=10, choose_better=True)
print(tune_lr_reg)

In [ ]:
interpret_model(tune_lr_reg)

In [ ]:
blend_all = blend_models()

In [ ]:
data["code_postal"] = data["code_postal"].apply(lambda x: f"{x}")

In [ ]:
data["code_postal"]

In [ ]:
pd.Series.apply()